In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
import threading
import multiprocessing
import time
import tensorflow as tf
drive.mount('/content/drive')
import tensorflow as tf
import os.path
tf.test.gpu_device_name()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'/device:GPU:0'

In [0]:
import os.path

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
train_data_b = train_data
calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 4 #スレッド数
process_num = 8 #プロセス数

current_num = [-1 for jj in range(thread_num + process_num)]  # 実行中のファイル番号(重複処理を防ぐ)

def make_tdb(meirei):
  exec(meirei)


def stap(ST):
        return calendar_data2["snap_" + ST]

def traindata2_making(th_num, df_name):
    for ii in range(start_n, len(train_data_b)):
        if ((ii in current_num) == False): #その行は処理されていない
            current_num[th_num] = ii
            if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_"+ df_name +"_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                train_data2a = train_data2a0.copy()
                tra_sir = train_data_b.iloc[ii]
                for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
                train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
                train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
                train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

                sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
                sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
                sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

                train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

                train_data2a["value"] = tra_sir[6:].to_list()

                train_data2a.to_csv(
                    "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_"+ df_name +"_" + str(ii) + ".csv", index=False)
            current_num[th_num] = -1

def combine_datas(df_name, meirei):
  make_tdb(meirei)
  thread_list = []
  with tf.device('/cpu:0'):
    for i in range(thread_num):
        thread = threading.Thread(target=traindata2_making, args=(i,df_name,))
        thread.start()
        thread_list.append(thread)
        print(i)
        sleep_seconds = 1
        time.sleep(sleep_seconds)

  with tf.device('/gpu:0'):
    for i2 in range(thread_num, thread_num + process_num):
        mp = threading.Thread(target=traindata2_making, args=(i2,df_name,))
        mp.start()
        thread_list.append(thread)
        print(i2)
        sleep_seconds = 1
        time.sleep(sleep_seconds)

  while(True):
    print('\r', end='')
    print(current_num, end='')
    sleep_seconds = 2
    time.sleep(sleep_seconds)

  for thread in thread_list:
      thread.join()

  print(df_name + "complete")

In [0]:
combine_datas("x2_data_CA_1","train_data_b = train_data[train_data[\"store_id\"] == \"CA_1\"]")
combine_datas("x2_data_CA_2","train_data_b = train_data[train_data[\"store_id\"] == \"CA_2\"]")
combine_datas("x2_data_CA_3","train_data_b = train_data[train_data[\"store_id\"] == \"CA_3\"]")
combine_datas("x2_data_TX_1","train_data_b = train_data[train_data[\"store_id\"] == \"TX_1\"]")
combine_datas("x2_data_TX_2","train_data_b = train_data[train_data[\"store_id\"] == \"TX_2\"]")
combine_datas("x2_data_TX_3","train_data_b = train_data[train_data[\"store_id\"] == \"TX_3\"]")
combine_datas("x2_data_WI_1","train_data_b = train_data[train_data[\"store_id\"] == \"WI_1\"]")
combine_datas("x2_data_WI_2","train_data_b = train_data[train_data[\"store_id\"] == \"WI_2\"]")
combine_datas("x2_data_WI_3","train_data_b = train_data[train_data[\"store_id\"] == \"WI_3\"]")

0
1
2
3
4
5
6
7
8
9
10
11
[7750, 7746, 7747, 7743, 7752, 7739, 7748, 7745, 7749, 7741, 7751, 7744]

In [0]:
combine_datas("x4_data_FOODS_1","train_data_b = train_data[train_data[\"dept_id\"] == \"x4_data_FOODS_1\"]")
combine_datas("x4_data_FOODS_2","train_data_b = train_data[train_data[\"dept_id\"] == \"x4_data_FOODS_2\"]")
combine_datas("x4_data_FOODS_3","train_data_b = train_data[train_data[\"dept_id\"] == \"x4_data_FOODS_3\"]")
combine_datas("x4_data_HOUSEHOLD_1","train_data_b = train_data[train_data[\"dept_id\"] == \"x4_data_HOUSEHOLD_1\"]")
combine_datas("x4_data_HOUSEHOLD_2","train_data_b = train_data[train_data[\"dept_id\"] == \"x4_data_HOUSEHOLD_2\"]")
combine_datas("x4_data_HOBBIES_1","train_data_b = train_data[train_data[\"dept_id\"] == \"x4_data_HOBBIES_1\"]")
combine_datas("x4_data_HOBBIES_2","train_data_b = train_data[train_data[\"dept_id\"] == \"x4_data_HOBBIES_2\"]")

In [0]:
import os.path
def df_combine(df_name):
  try:
    main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv")
    with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt") as f:
        ii = int(f.read())
  except:
    ii = 1
  finally:
    error_flag = False

    while True:
      try:
        df = pd.read_csv"/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_"+ df_name +"_" + str(ii) + ".csv")
      except:
        print(str(ii)+ "doesn't exist")
        error_flag = True
        if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False): berak
      else:
        main_data = pd.concat([main_data, df])
        print('\r', end='')
        print(ii, end='')
        if (ii%100 == 0):
          if (error_flag == True): sys.exit(1)
          with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt", mode='w') as f:
            f.write(str(ii))
          main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv", index=False)
      finally:
        ii += 1

    main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv", index=False)
    os.remove("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt")
    os.remove("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv")

In [0]:
df_combine("x2_data_CA_1")
df_combine("x2_data_CA_2")
df_combine("x2_data_CA_3")
df_combine("x2_data_TX_1")
df_combine("x2_data_TX_2")
df_combine("x2_data_TX_3")
df_combine("x2_data_WI_1")
df_combine("x2_data_WI_2")
df_combine("x2_data_WI_3")
df_combine("x4_data_FOODS_1")
df_combine("x4_data_FOODS_2")
df_combine("x4_data_FOODS_3")
df_combine("x4_data_HOUSEHOLD_1")
df_combine("x4_data_HOUSEHOLD_2")
df_combine("x4_data_HOBBIES_1")
df_combine("x4_data_HOBBIES_2")

In [0]:
ii = 1413
if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
  train_data2a = train_data2a0.copy()
  tra_sir = train_data.iloc[ii]
  for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
  train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
  train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
  train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
  train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

  sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
  sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
  sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

  train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

  train_data2a["value"] = tra_sir[6:].to_list()

  train_data2a.to_csv(
      "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv", index=False)
else: print("files exist")

各種データを学習

In [0]:
import pickle
import lightgbm as lgb
import gc
x_list = ["x1_data_CA","x1_data_TX","x1_data_WI","x2_data_CA_1","x2_data_CA_2","x2_data_CA_3","x2_data_TX_1","x2_data_TX_2","x2_data_TX_3","x2_data_WI_1","x2_data_WI_2","x2_data_WI_3","x3_data_FOODS","x3_data_HOUSEHOLD","x3_data_HOBBIES","x4_data_FOODS_1","x4_data_FOODS_2","x4_data_FOODS_3","x4_data_HOUSEHOLD_1","x4_data_HOUSEHOLD_2","x4_data_HOBBIES_1","x4_data_HOBBIES_2"]
in_cols = ["wday","month","year","Religious","National","Cultural","Sporting","snap","sell_price","price_down","price_up"]
categorical_features = ["wday","month","year","Religious","National","Cultural","Sporting","snap","price_down","price_up"] #販売価格以外をカテゴリ変数化
lgb_params = {"objective":"regression", 
              "metric":"rmse", 
              "verbosity": -1}

for tt in x_list:
  mdax = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/" + tt + ".csv")
  train_input = mdax[in_cols]
  train_result = mdax["value"]
  train_input = train_input.astype('category')

  train_datas = lgb.Dataset(train_input, train_result)

  clf = lgb.train(lgb_params, train_datas)

  file = 'trained_model_' + tt + '.pkl'
  pickle.dump(clf, open(file, 'wb'))

  del clf
  gc.collect()

In [0]:
import gc
mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all.csv")

test_data1886_1913 = mda[mda['d'] >= 1886]
test_data1886_1913 = test_data1886_1913[test_data1886_1913['d'] <= 1913]
test_data1886_1913.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/test_data1886_1913.csv", index=False)